https://learn.deeplearning.ai/langchain-chat-with-your-data

In [1]:
import chatbot.properties as properties


#### Loading and Chunking Input Documents

In [2]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

directory = DirectoryLoader(
    "docs/", glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader
)
pages = directory.load()

100%|██████████| 42/42 [00:22<00:00,  1.86it/s]


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 400
chunk_overlap = 200

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    separators= ["\n\n", "\n", "(?<=\. )", " ", ]
)

docs = r_splitter.split_documents(pages)
len(docs)

2528

#### Embedding and VectorDB

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(openai_api_key=properties.OPENAI_KEY)
persist_directory = "chroma/"

In [5]:
# new db from documents

vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

vectordb.persist()

#### Retrieving relevant chunks

In [3]:
# reopen existing db

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [39]:
question = "Was ist die Haltung der Autorin gegenüber Menschen in Indien, Nepal,  Rwanda, Tschad, Persien oder USA?"
# "Sind Alf und Margrit rassistisch?"
# "Was ist die Haltung der Autorin zu Religion?"
# "Wo auf der Welt haben Margrit und Alf de Spindler Freunde oder Verwandte?"
# "Welche Orte auf diese Welt haben die Autorin und ihr Ehemann Alf besucht?"
# "Was sind die Stärken von Olaf, Irène, Christine, Thérèse?"
# "Was sind die Stärken von Olaf, Irène, Christine, Thérèse?"
# "Welche Konflikte gibt es in der Familie?"
# "Gibt es Anzeichen von Demenz in der Familie?"
# "Was wurde in Nepal erlebt?"

# "Was macht Alexandre?" 
# "Was würde Alf auf die Frage 'wie geht es dir?' antworten?"

In [ ]:
docs = vectordb.similarity_search(question, k=10)

In [43]:
docs = vectordb.max_marginal_relevance_search(question, fetch_k=20, k=10)

In [44]:
print(docs)

AttributeError: 'list' object has no attribute 'len'

#### Using GPT for question answering

In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=properties.OPENAI_KEY, model_name="gpt-4", temperature=0)

In [41]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
result = qa_chain({"query": question})
result["result"]

'Der Text gibt keine spezifische Information über die Haltung der Autorin gegenüber Menschen in Indien, Nepal, Rwanda, Tschad, Persien oder USA.'

#### Using GPT for task generation

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Erzeuge eine Single-Choice-Aufgabe mit 4 Antwortoptionen, wovon eine richtig ist. Die Aufgabe soll folgendes Wissen prüfen: 
{context}
Single-Choice-Aufgabe:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Baltimore"
result = qa_chain({"query": question})
result["source_documents"]

In [ ]:
result["result"]